In [5]:
def parse_three_address_code(input_text):
    """Parse input text into blocks of three address code instructions."""
    blocks = {}
    current_block_id = None
    instruction_counter = 1

    lines = input_text.strip().split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("Block"):
            # Extract block id from "Block X" format
            try:
                current_block_id = int(line.split()[1])
                blocks[current_block_id] = []
            except (IndexError, ValueError):
                print(f"Warning: Invalid block header: {line}")
                continue
        elif current_block_id is not None:
            # Add instruction to current block
            blocks[current_block_id].append((instruction_counter, line))
            instruction_counter += 1

    return blocks

def build_control_flow_graph(blocks):
    """Build a control flow graph from blocks of three address code."""
    cfg = {}

    # Initialize with empty successors and predecessors
    for block_id in blocks:
        cfg[block_id] = {'succ': set(), 'pred': set()}

    # Analyze each block for control flow
    for block_id, instructions in blocks.items():
        if not instructions:
            continue

        # Get the last instruction of the block
        last_instr = instructions[-1][1]

        # Check if it's a conditional branch
        if last_instr.startswith('if') and 'goto' in last_instr:
            # Extract target block from "if condition goto Block X"
            try:
                target_parts = last_instr.split('goto')
                if len(target_parts) > 1:
                    target_block = int(target_parts[1].strip().split()[1])
                    # Add edge from current block to target block
                    cfg[block_id]['succ'].add(target_block)
                    cfg[target_block]['pred'].add(block_id)

                    # Also add fallthrough edge to next block unless it's the last block
                    next_block = block_id + 1
                    if next_block in blocks:
                        cfg[block_id]['succ'].add(next_block)
                        cfg[next_block]['pred'].add(block_id)
            except (IndexError, ValueError) as e:
                print(f"Warning: Error parsing branch in block {block_id}: {e}")

        # Check if it's an unconditional jump
        elif last_instr.startswith('goto'):
            # Extract target block from "goto Block X"
            try:
                target_block = int(last_instr.split()[1].strip())
                # Add edge from current block to target block
                cfg[block_id]['succ'].add(target_block)
                cfg[target_block]['pred'].add(block_id)
            except (IndexError, ValueError) as e:
                print(f"Warning: Error parsing jump in block {block_id}: {e}")

        # If no jump or branch, add fallthrough edge to next block
        elif block_id + 1 in blocks:
            cfg[block_id]['succ'].add(block_id + 1)
            cfg[block_id + 1]['pred'].add(block_id)

    return cfg

def compute_gen_kill(blocks):
    """Compute GEN and KILL sets for each block of three address code."""
    gen_kill = {}

    for block_id, instructions in blocks.items():
        gen_set = set()  # Variables used before defined in the block (by instruction number)
        kill_set = set()  # Variables defined in the block (by instruction number)
        defined_in_block = set()  # Track variables defined within this block

        for instruction_no, instruction in instructions:
            parts = instruction.split()

            # Skip non-assignments and control flow instructions
            if len(parts) < 3 or parts[1] != '=' or instruction.startswith(('if', 'goto')):
                continue

            # Extract target variable (left-hand side)
            target = parts[0]
            defined_in_block.add(target)
            kill_set.add(instruction_no)

            # Extract used variables (right-hand side)
            for part in parts[2:]:
                # If it's not an operator or constant, it's a variable
                if not part.isdigit() and part not in ['+', '-', '*', '/', '%', '(', ')', ',', '>=', '<=', '<', '>', '==']:
                    # Only add to GEN if not already defined in this block
                    if part not in defined_in_block:
                        gen_set.add(instruction_no)

        gen_kill[block_id] = {'GEN': gen_set, 'KILL': kill_set}

    return gen_kill

def perform_in_out_analysis(blocks, cfg, gen_kill):
    """
    Perform IN-OUT analysis for each block using the control flow graph.
    Returns a dictionary with IN and OUT sets for each block.
    """
    # Initialize IN and OUT sets
    analysis = {}
    for block_id in blocks:
        analysis[block_id] = {'IN': set(), 'OUT': set()}

    # Iteratively compute IN and OUT sets until convergence
    iteration = 1
    changed = True

    while changed:
        changed = False
        print(f"\nIteration {iteration}:")

        for block_id in sorted(blocks.keys()):
            # Save old values for comparison
            old_in = analysis[block_id]['IN'].copy()
            old_out = analysis[block_id]['OUT'].copy()

            # Compute IN[i] = ∪ OUT[j] for all predecessors j of i
            in_set = set()
            for pred in cfg[block_id]['pred']:
                in_set = in_set.union(analysis[pred]['OUT'])
            analysis[block_id]['IN'] = in_set

            # Compute OUT[i] = GEN[i] ∪ (IN[i] - KILL[i])
            out_set = gen_kill[block_id]['GEN'].union(
                analysis[block_id]['IN'] - gen_kill[block_id]['KILL']
            )
            analysis[block_id]['OUT'] = out_set

            # Print current state
            print(f"  Block {block_id}:")
            print(f"    IN:  {', '.join(map(str, sorted(analysis[block_id]['IN']))) if analysis[block_id]['IN'] else 'Ø'}")
            print(f"    OUT: {', '.join(map(str, sorted(analysis[block_id]['OUT']))) if analysis[block_id]['OUT'] else 'Ø'}")

            # Check if anything changed
            if old_in != analysis[block_id]['IN'] or old_out != analysis[block_id]['OUT']:
                changed = True

        iteration += 1

    # Add GEN and KILL to the final analysis
    for block_id in blocks:
        analysis[block_id]['GEN'] = gen_kill[block_id]['GEN']
        analysis[block_id]['KILL'] = gen_kill[block_id]['KILL']

    return analysis

def detect_loop_invariants(cfg, blocks):
    """
    Detect loop invariants by finding expressions that do not change within loops.
    """
    loop_invariants = {}

    # Analyze the CFG for loops (simple version based on back edges)
    for block_id in blocks:
        for succ in cfg[block_id]['succ']:
            # Check for back edges (loops)
            if succ in cfg[block_id]['pred']:
                # If a back edge exists, we have a loop
                print(f"\nFound loop between Block {block_id} and Block {succ}")
                loop_invariants[block_id] = set()
                for instruction_no, instruction in blocks[block_id]:
                    parts = instruction.split()
                    print(f"  Analyzing instruction {instruction_no}: {instruction}")

                    # Detect loop invariants by looking for expressions that don't change
                    if len(parts) >= 3 and parts[1] == '=':
                        lhs = parts[0]
                        rhs = parts[2:]

                        # Check if rhs has variables that depend on loop variables
                        is_invariant = True
                        for var in rhs:
                            # If rhs contains any loop-dependent variables (like 'i'), it's not invariant
                            if var in ['i', 'n']:  # You can extend this list for more loop variables
                                is_invariant = False
                                break

                        if is_invariant:
                            print(f"    Loop Invariant detected: {lhs} = {''.join(rhs)}")
                            loop_invariants[block_id].add(lhs)
                        else:
                            print(f"    {lhs} is not a loop invariant because it depends on variables that change in the loop.")

    return loop_invariants




def main():
    print("Three Address Code Analysis with Blocks, Instruction Sets, and Loop Invariants")
    print("Enter your code blocks (format: 'Block X' followed by instructions, enter 'done' when finished):")

    input_lines = []
    while True:
        line = input()
        if line.lower() == 'done':
            break
        input_lines.append(line)

    input_text = '\n'.join(input_lines)
    blocks = parse_three_address_code(input_text)

    print("\nParsed Blocks:")
    for block_id in sorted(blocks.keys()):
        print(f"Block {block_id}:")
        for instruction_no, instruction in blocks[block_id]:
            print(f"  Instruction {instruction_no}: {instruction}")

    # Build control flow graph
    cfg = build_control_flow_graph(blocks)

    # Compute GEN and KILL sets for each block
    gen_kill = compute_gen_kill(blocks)

    # Perform IN-OUT analysis
    analysis = perform_in_out_analysis(blocks, cfg, gen_kill)

    # Detect loop invariants
    loop_invariants = detect_loop_invariants(cfg, blocks)

    # Output the results
    print("\nFinal IN-OUT Analysis:")
    for block_id in sorted(analysis.keys()):
        print(f"Block {block_id}:")
        print(f"  GEN:  {', '.join(map(str, sorted(analysis[block_id]['GEN']))) if analysis[block_id]['GEN'] else 'Ø'}")
        print(f"  KILL: {', '.join(map(str, sorted(analysis[block_id]['KILL']))) if analysis[block_id]['KILL'] else 'Ø'}")
        print(f"  IN:   {', '.join(map(str, sorted(analysis[block_id]['IN']))) if analysis[block_id]['IN'] else 'Ø'}")
        print(f"  OUT:  {', '.join(map(str, sorted(analysis[block_id]['OUT']))) if analysis[block_id]['OUT'] else 'Ø'}")

    print("\nDetected Loop Invariants: Ø")
    for block_id, invariants in loop_invariants.items():
        print(f"Block {block_id}: Loop Invariants - {', '.join(map(str, sorted(invariants)))}")

if __name__ == "__main__":
    main()

Three Address Code Analysis with Blocks, Instruction Sets, and Loop Invariants
Enter your code blocks (format: 'Block X' followed by instructions, enter 'done' when finished):
Block 1
i = 0
sum = 0

Block 2
t1 = 4 * i
t2 = i * 10
if i >= 10 goto Block 4

Block 3
sum = sum + t2
i = i + 1
goto Block 2

Block 4
return
done

Parsed Blocks:
Block 1:
  Instruction 1: i = 0
  Instruction 2: sum = 0
Block 2:
  Instruction 3: t1 = 4 * i
  Instruction 4: t2 = i * 10
  Instruction 5: if i >= 10 goto Block 4
Block 3:
  Instruction 6: sum = sum + t2
  Instruction 7: i = i + 1
  Instruction 8: goto Block 2
Block 4:
  Instruction 9: return

Iteration 1:
  Block 1:
    IN:  Ø
    OUT: Ø
  Block 2:
    IN:  Ø
    OUT: 3, 4
  Block 3:
    IN:  3, 4
    OUT: 3, 4, 6
  Block 4:
    IN:  3, 4
    OUT: 3, 4

Iteration 2:
  Block 1:
    IN:  Ø
    OUT: Ø
  Block 2:
    IN:  Ø
    OUT: 3, 4
  Block 3:
    IN:  3, 4
    OUT: 3, 4, 6
  Block 4:
    IN:  3, 4
    OUT: 3, 4

Final IN-OUT Analysis:
Block 1:
  GEN:

Input

Block 1
i = 0
sum = 0

Block 2
t1 = 4 * i
t2 = i * 10
if i >= 10 goto Block 4

Block 3
sum = sum + t2
i = i + 1
goto Block 2

Block 4
return
done